In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, date, timedelta
import os

# Scikit-learn for preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import IsolationForest # For outlier detection

In [2]:
# --- Configuration ---
# Define the path to your locally available historical data CSV file.
LOCAL_DATA_FILE = 'complete_history_store1.csv'

# Define the output path for the processed data
PROCESSED_DATA_FILE = 'processed_complete_history_store1.csv'

# Define the date range for processing (should match your historical data availability)
DATA_START_DATE = datetime(2024, 1, 1, 0, 0, 0) # Start of your data
DATA_END_DATE = datetime(2025, 6, 13, 23, 59, 59) # End of your data (or latest date available)

In [3]:
# --- 1. Load Data ---
print(f"--- Loading Data from {LOCAL_DATA_FILE} ---")
try:
    if not os.path.exists(LOCAL_DATA_FILE):
        raise FileNotFoundError(f"Local data file '{LOCAL_DATA_FILE}' not found. Please ensure it's in the correct directory.")

    df = pd.read_csv(LOCAL_DATA_FILE)

    if df.empty:
        raise ValueError("Loaded DataFrame is empty.")

    # Ensure timestamp is datetime and filter by date range
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # Filter DataFrame by the specified date range
    df = df[(df['timestamp'] >= DATA_START_DATE) & (df['timestamp'] <= DATA_END_DATE)]

    if df.empty:
        print(f"No data found within the specified date range: {DATA_START_DATE} to {DATA_END_DATE}.")
        exit() # Exit if no data for the specified range

    # Set timestamp as index for time series operations
    df = df.set_index('timestamp')

    # Ensure the index is sorted chronologically
    df = df.sort_index()

    print(f"Successfully loaded {len(df)} rows for preprocessing within the specified range.")
    print("DataFrame Head (initial load):")
    print(df.head().to_string())

except Exception as e:
    print(f"Failed to load historical data from local file: {e}")
    exit()

--- Loading Data from complete_history_store1.csv ---
Successfully loaded 12720 rows for preprocessing within the specified range.
DataFrame Head (initial load):
                     HVAC_Energy(t)  Refrigeration_Energy(t)  Lighting_Energy(t)  IT_System(t)  Other_System(t)  Electricity_Price(t)  Solar_Available_for_Use(t)  Solar_Used_to_Charge_Battery(t)  Base_Consumption_Loads(t)  Total_Energy(t)  Battery_1_Charge_Discharge(t)  Battery_1_Energy_Stored(t)  Battery_2_Charge_Discharge(t)  Battery_2_Energy_Stored(t)
timestamp                                                                                                                                                                                                                                                                                                                                                           
2024-01-01 00:00:00      112.630998               191.979173            0.845627     66.751797        51.582381             

In [4]:
# --- 2. Feature Engineering ---
print("\n--- Starting Feature Engineering ---")

# --- 2.1 Time-based Features ---
df['hour_of_day'] = df.index.hour
df['day_of_week'] = df.index.dayofweek # Monday=0, Sunday=6
df['day_of_month'] = df.index.day
df['day_of_year'] = df.index.dayofyear
df['week_of_year'] = df.index.isocalendar().week.astype(int)
df['month'] = df.index.month
df['quarter'] = df.index.quarter
df['is_weekend'] = (df.index.dayofweek >= 5).astype(int) # 1 if Sat/Sun, 0 otherwise

# Simple season mapping (numeric encoding for models)
def get_season_numeric(month):
    if 3 <= month <= 5: return 0 # Spring
    elif 6 <= month <= 8: return 1 # Summer
    elif 9 <= month <= 11: return 2 # Autumn
    else: return 3 # Winter
df['season_encoded'] = df['month'].apply(get_season_numeric)

print("Added comprehensive time-based features.")

# --- 2.2 Lagged Values ---
# Lagged features are crucial for time series forecasting.
lag_cols = ['Total_Energy(t)', 'Solar_Available_for_Use(t)', 'Electricity_Price(t)',
            'HVAC_Energy(t)', 'Refrigeration_Energy(t)', 'Lighting_Energy(t)']
lags = [1, 2, 3, 24, 24*7] # Lag by 1, 2, 3 hours, 1 day, 7 days

for col in lag_cols:
    for lag in lags:
        df[f'{col}_lag_{lag}h'] = df[col].shift(lag)
print(f"Generated lagged features for {', '.join(lag_cols)} for lags: {lags}.")

# --- 2.3 Rolling Averages ---
# Rolling features help capture trends and seasonality.
rolling_cols = ['Total_Energy(t)', 'Solar_Available_for_Use(t)', 'Electricity_Price(t)']
windows = [3, 6, 24, 24*7] # Rolling window of 3, 6 hours, 1 day, 7 days

for col in rolling_cols:
    for window in windows:
        df[f'{col}_roll_mean_{window}h'] = df[col].rolling(window=window, min_periods=1).mean()
        df[f'{col}_roll_std_{window}h'] = df[col].rolling(window=window, min_periods=1).std()
print(f"Generated rolling mean and standard deviation features for {', '.join(rolling_cols)} for windows: {windows}.")

# --- 2.4 Interaction Features ---
# Combine features to capture more complex relationships.
df['Consumption_x_Price'] = df['Total_Energy(t)'] * df['Electricity_Price(t)']
df['Solar_x_DaylightHours'] = df['Solar_Available_for_Use(t)'] * (df['hour_of_day'].apply(lambda x: 1 if 6 <= x <= 18 else 0))
df['HVAC_x_IsWeekend'] = df['HVAC_Energy(t)'] * df['is_weekend']
df['Fridge_x_IsWeekend'] = df['Refrigeration_Energy(t)'] * df['is_weekend']
print("Generated interaction features.")

# --- 2.5 Holiday Indicators (Placeholder) ---
# For synthetic data, we can define a few dummy holidays.
# For real-world data, use a robust holiday library like `holidays`.
df['is_holiday'] = 0 # Default to no holiday
# Example: Christmas and New Year's Day
df.loc[df.index.normalize() == pd.to_datetime('2024-01-01').normalize(), 'is_holiday'] = 1
df.loc[df.index.normalize() == pd.to_datetime('2024-12-25').normalize(), 'is_holiday'] = 1
df.loc[df.index.normalize() == pd.to_datetime('2025-01-01').normalize(), 'is_holiday'] = 1
df.loc[df.index.normalize() == pd.to_datetime('2025-12-25').normalize().date(), 'is_holiday'] = 1 # Using .date() for comparison
print("Added placeholder 'is_holiday' feature.")


--- Starting Feature Engineering ---
Added comprehensive time-based features.
Generated lagged features for Total_Energy(t), Solar_Available_for_Use(t), Electricity_Price(t), HVAC_Energy(t), Refrigeration_Energy(t), Lighting_Energy(t) for lags: [1, 2, 3, 24, 168].
Generated rolling mean and standard deviation features for Total_Energy(t), Solar_Available_for_Use(t), Electricity_Price(t) for windows: [3, 6, 24, 168].
Generated interaction features.
Added placeholder 'is_holiday' feature.


In [5]:
# --- 3. Noise Reduction (Simple Smoothing) ---
print("\n--- Applying Noise Reduction (Rolling Mean Smoothing) ---")
# Apply a rolling mean to smooth primary consumption/generation features.
# This helps reduce high-frequency noise without losing too much information.
smoothing_cols = ['HVAC_Energy(t)', 'Refrigeration_Energy(t)', 'Lighting_Energy(t)',
                  'IT_System(t)', 'Other_System(t)', 'Solar_Available_for_Use(t)',
                  'Electricity_Price(t)']
smoothing_window = 3 # 3-hour rolling mean for smoothing

for col in smoothing_cols:
    df[f'{col}_smoothed'] = df[col].rolling(window=smoothing_window, min_periods=1, center=True).mean()
    # Replace original with smoothed if preferred, or keep both
    df[col] = df[f'{col}_smoothed']
    df.drop(columns=[f'{col}_smoothed'], inplace=True) # Remove temporary smoothed column
print(f"Applied {smoothing_window}-hour rolling mean smoothing to key energy columns.")



--- Applying Noise Reduction (Rolling Mean Smoothing) ---
Applied 3-hour rolling mean smoothing to key energy columns.


In [6]:
# --- 4. Outlier Handling ---
print("\n--- Handling Outliers using IsolationForest ---")
# Isolation Forest is an unsupervised algorithm suitable for outlier detection.
# It works well for high-dimensional datasets and is robust to noise.

# Select numerical columns for outlier detection (excluding one-hot/binary encoded)
# Also exclude battery stored levels if they are expected to hit min/max frequently
outlier_cols = [
    'Total_Energy(t)', 'Solar_Available_for_Use(t)', 'Electricity_Price(t)',
    'HVAC_Energy(t)', 'Refrigeration_Energy(t)', 'Lighting_Energy(t)',
    'IT_System(t)', 'Other_System(t)', 'Battery_1_Charge_Discharge(t)',
    'Battery_2_Charge_Discharge(t)'
]

# Filter to only include columns that exist after smoothing
outlier_cols = [col for col in outlier_cols if col in df.columns]

if outlier_cols:
    # Initialize IsolationForest. contamination is the estimated proportion of outliers in the data.
    # Adjust contamination based on your data's expected outlier frequency.
    iso_forest = IsolationForest(random_state=42, contamination=0.01) # Assuming 1% outliers

    # Fit and predict outliers. -1 for outliers, 1 for inliers.
    df['outlier_score'] = iso_forest.fit_predict(df[outlier_cols])

    # Option 1: Remove outliers (common for training data)
    initial_rows_before_outlier_removal = len(df)
    df = df[df['outlier_score'] == 1].copy() # Keep only inliers
    print(f"Removed {initial_rows_before_outlier_removal - len(df)} outlier rows.")

    # Option 2 (Alternative: Impute outliers instead of removing):
    # If you prefer to impute, you might replace outliers with the median/mean of the feature
    # or with a boundary value (e.g., 99th percentile).
    # For example: df.loc[df['outlier_score'] == -1, outlier_cols] = df[outlier_cols].median().values

    df.drop(columns=['outlier_score'], inplace=True) # Drop the temporary outlier score column
else:
    print("No suitable columns for outlier detection found or selected.")


# --- Handle NaNs generated by lagging/rolling operations ---
# After generating lagged and rolling features, the first few rows will have NaNs.
# It's usually best to drop these for model training.
initial_rows_before_nan_drop = len(df)
df.dropna(inplace=True)
print(f"Dropped {initial_rows_before_nan_drop - len(df)} rows with NaN values (from lagging/rolling).")
print(f"DataFrame now has {len(df)} rows after cleaning.")


--- Handling Outliers using IsolationForest ---
Removed 128 outlier rows.
Dropped 161 rows with NaN values (from lagging/rolling).
DataFrame now has 12431 rows after cleaning.


In [7]:
# --- 5. Column Management ---
print("\n--- Managing Columns for Model Training ---")

# Define target variable
TARGET_COLUMN = 'Total_Energy(t)'

# Columns to keep for features (X)
# Include most engineered features and original ones that are relevant.
# Exclude original 'timestamp' (it's the index), and the raw values if smoothed versions are used.
# Exclude Battery_X_Energy_Stored(t) if they are considered "state" outputs or highly correlated with other features.
# Make sure to include the lagged values of the target variable for autoregressive models.

# Start with all current columns
features_to_keep = list(df.columns)

# Remove the target variable itself from features
if TARGET_COLUMN in features_to_keep:
    features_to_keep.remove(TARGET_COLUMN)

# Remove non-numeric/redundant time columns if they aren't directly used as features
columns_to_drop_post_engineering = [
    'day_name', 'month_name', # String representations
    'Base_Consumption_Loads(t)', # Derived directly from other consumption loads, can be redundant
    # Potentially remove some original raw columns if smoothed versions are used effectively
]
features_to_keep = [col for col in features_to_keep if col not in columns_to_drop_post_engineering]

# Ensure specific critical columns are included (e.g., battery stored levels if they are inputs for forecasting next consumption)
# Re-add specific columns if they were accidentally dropped but are needed
# Example: if you plan to predict 'Total_Energy(t)', then 'Battery_X_Energy_Stored(t)' from previous hour might be crucial features.
# For simplicity, let's keep all numerical columns that aren't being dropped explicitly.

# Final list of features (X) and target (y)
X = df[features_to_keep]
y = df[TARGET_COLUMN]

print(f"Features (X) shape: {X.shape}")
print(f"Target (y) shape: {y.shape}")
print("Sample Features after column management:")
print(X.head().to_string())


--- Managing Columns for Model Training ---
Features (X) shape: (12431, 80)
Target (y) shape: (12431,)
Sample Features after column management:
                     HVAC_Energy(t)  Refrigeration_Energy(t)  Lighting_Energy(t)  IT_System(t)  Other_System(t)  Electricity_Price(t)  Solar_Available_for_Use(t)  Solar_Used_to_Charge_Battery(t)  Battery_1_Charge_Discharge(t)  Battery_1_Energy_Stored(t)  Battery_2_Charge_Discharge(t)  Battery_2_Energy_Stored(t)  hour_of_day  day_of_week  day_of_month  day_of_year  week_of_year  month  quarter  is_weekend  season_encoded  Total_Energy(t)_lag_1h  Total_Energy(t)_lag_2h  Total_Energy(t)_lag_3h  Total_Energy(t)_lag_24h  Total_Energy(t)_lag_168h  Solar_Available_for_Use(t)_lag_1h  Solar_Available_for_Use(t)_lag_2h  Solar_Available_for_Use(t)_lag_3h  Solar_Available_for_Use(t)_lag_24h  Solar_Available_for_Use(t)_lag_168h  Electricity_Price(t)_lag_1h  Electricity_Price(t)_lag_2h  Electricity_Price(t)_lag_3h  Electricity_Price(t)_lag_24h  Electricity_

In [8]:
# --- 6. Data Scaling ---
print("\n--- Scaling Data for Model Training ---")
# Scaling is essential for many ML algorithms (e.g., neural networks, SVMs, linear models, k-NN).
# It prevents features with larger values from dominating the learning process.

# Identify numerical columns to scale. Exclude binary/categorical features.
numerical_cols = X.select_dtypes(include=np.number).columns.tolist()
# Exclude binary features like 'is_weekend', 'is_holiday' from scaling if you want them as 0/1
# Adjust based on your specific engineered features
cols_to_exclude_from_scaling = ['is_weekend', 'is_holiday']
numerical_cols_to_scale = [col for col in numerical_cols if col not in cols_to_exclude_from_scaling]

# Use StandardScaler for most cases (mean=0, variance=1)
scaler_X = StandardScaler()
# Fit and transform features
X_scaled = X.copy() # Create a copy to avoid modifying original X
X_scaled[numerical_cols_to_scale] = scaler_X.fit_transform(X[numerical_cols_to_scale])

# It's also common to scale the target variable, especially for regression tasks
# if its range is very large or for neural networks.
scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1)).flatten() # Reshape for scaler, then flatten back

print("Numerical features in X scaled using StandardScaler.")
print("Target variable (y) scaled using StandardScaler.")
print("\nScaled Features (X_scaled) Head:")
print(X_scaled.head().to_string())
print("\nScaled Target (y_scaled) Head:")
print(y_scaled[:5]) # Print first 5 scaled target values


--- Scaling Data for Model Training ---
Numerical features in X scaled using StandardScaler.
Target variable (y) scaled using StandardScaler.

Scaled Features (X_scaled) Head:
                     HVAC_Energy(t)  Refrigeration_Energy(t)  Lighting_Energy(t)  IT_System(t)  Other_System(t)  Electricity_Price(t)  Solar_Available_for_Use(t)  Solar_Used_to_Charge_Battery(t)  Battery_1_Charge_Discharge(t)  Battery_1_Energy_Stored(t)  Battery_2_Charge_Discharge(t)  Battery_2_Energy_Stored(t)  hour_of_day  day_of_week  day_of_month  day_of_year  week_of_year     month   quarter  is_weekend  season_encoded  Total_Energy(t)_lag_1h  Total_Energy(t)_lag_2h  Total_Energy(t)_lag_3h  Total_Energy(t)_lag_24h  Total_Energy(t)_lag_168h  Solar_Available_for_Use(t)_lag_1h  Solar_Available_for_Use(t)_lag_2h  Solar_Available_for_Use(t)_lag_3h  Solar_Available_for_Use(t)_lag_24h  Solar_Available_for_Use(t)_lag_168h  Electricity_Price(t)_lag_1h  Electricity_Price(t)_lag_2h  Electricity_Price(t)_lag_3h  Electr

In [9]:
# --- 7. Generate Processed Data in CSV Format ---
print("\n--- Saving Processed Data ---")

# Combine scaled features and scaled target back into a single DataFrame for saving
# Add the target variable back as a column
processed_df = X_scaled.copy()
processed_df[TARGET_COLUMN] = y_scaled

# Save to CSV
processed_df.to_csv(PROCESSED_DATA_FILE, index=True) # Keep timestamp index in CSV
print(f"Processed data saved to: {PROCESSED_DATA_FILE}")
print(f"Shape of processed data: {processed_df.shape}")
print("Sample of processed data (first 5 rows and selected columns):")
print(processed_df.head()[['Total_Energy(t)', 'Total_Energy(t)_lag_24h', 'Solar_Available_for_Use(t)_roll_mean_24h']].to_string())

print("\n--- Preprocessing Complete ---")


--- Saving Processed Data ---
Processed data saved to: processed_complete_history_store1.csv
Shape of processed data: (12431, 81)
Sample of processed data (first 5 rows and selected columns):
                     Total_Energy(t)  Total_Energy(t)_lag_24h  Solar_Available_for_Use(t)_roll_mean_24h
timestamp                                                                                              
2024-01-08 00:00:00        -1.527204                -1.490279                                 -0.273293
2024-01-08 01:00:00        -1.545085                -1.625451                                 -0.273293
2024-01-08 02:00:00        -1.642142                -1.871750                                 -0.273293
2024-01-08 03:00:00        -1.739039                -1.606841                                 -0.273293
2024-01-08 04:00:00        -1.753226                -1.666412                                 -0.273293

--- Preprocessing Complete ---
